# Preparing the labels in our DataFrame

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [2]:
if os.path.exists('data/raw_df'):
    df = pd.read_pickle('data/raw_df')
else:
    print('Please run the notebook for generating the raw dataframe first!')

In [3]:
df.head()

,img_url,room_type,room_type_list
0,https://blok-production.imgix.net/photos/e7a3e...,Kylpyhuone,[Kylpyhuone]
1,https://blok-production.imgix.net/photos/e7a3e...,Kylpyhuone,[Kylpyhuone]
2,https://blok-production.imgix.net/photos/e7a3e...,Pohjakuva,[Pohjakuva]
3,https://blok-production.imgix.net/photos/e7a3e...,Keittiö,[Keittiö]
4,https://blok-production.imgix.net/photos/e7a3e...,Keittiö,[Keittiö]


## Starting working on the data:

Extracting all the existing labels:

In [4]:
room_names = pd.Series([e for l in df.room_type_list.values for e in l])

In [5]:
unique = room_names.unique()  # all the roomnames that are in the list
unique.sort()
unique

array([' Erillinen wc', ' Piha', ' Sauna', ' Talo', '& Piha',
       'Alakerran makuuhuoneet', 'Alakerran oleskelutila',
       'Alakerran suihkut', 'Alakerran wc ja kodinhoitohuone', 'Alakerta',
       'Alempi kerros', 'Alkovi', 'Askarteluhuone', 'Asunto', 'Aula',
       'Autohalli', 'Autotalli', 'Avokeittiö', 'Erillinen WC',
       'Erillinen Wc', 'Erillinen kylpyhuone', 'Erillinen wc',
       'Erillinen wc ja kodinhoitohuone', 'Erilliset WC:t',
       'Erilliset wc:t', 'Eteinen', 'Eteinen ja keittiö',
       'Eteinen ja kylpyhuone', 'Eteinen ja makuuhuoneet',
       'Eteinen ja olohuone', 'Eteinen, olohuone',
       'Eteinen, olohuone ja makuutila', 'Etu- ja Takapiha', 'Etupiha',
       'Huone', 'Huoneet', 'Ilmakuvat', 'Julkisivu', 'Julkisivu ja piha',
       'Julkisivu, Maisema', 'Julkisivu, terassi', 'Julkisivut', 'Katto',
       'Kattoterassi', 'Keittiö', 'Keittiö ja alakerran kylpyhuone',
       'Keittiö ja kylpyhuone', 'Keittiö ja ruokailutila', 'Keittokomero',
       'Kellari'

In [6]:
room_names.value_counts()[room_names.value_counts() > 20]

Olohuone               3131
Keittiö                2453
Eteinen                2150
Kylpyhuone             1866
Parveke                1822
Talo                   1812
Makuuhuoneet           1390
Julkisivu              1346
Sauna                  1205
Terassi                1003
Pohjakuva               856
Erillinen WC            784
Ruokailutila            731
Makuuhuone              603
Piha                    478
Avokeittiö              190
Yläkerta                167
Alakerta                136
Erillinen wc            128
Vaatehuone              122
Huone                   118
Kodinhoitohuone         106
yläkerta                 83
Olohuone ja keittiö      67
Takkahuone               65
Alkovi                   61
Lasitettu parveke        47
Erilliset WC:t           46
WC                       43
Kylpyhuoneet             43
Eteinen ja olohuone      26
Kylpyhuone ja sauna      25
Kylphuone                25
Parvekkeet               25
Keittokomero             24
sauna               

The above looks way too messy.

Process label strings in df.room_type_list:
- lower() & split strings at delimiters of any kind or spaces (thereby also removing the delimiters and spaces)  
- Since 'erilliset wc'/'erillinen wc' seems to be a frequent term put those two words together again (if applicable replacing 'erilliset' with 'erillinen')
- some words appear with several different endings, remove special endings
- remove some obvious typos like 'Kylphuone' instead of 'Kylpyhuone'
- remove 't's
- change every kind of sauna to just 'sauna' (and do the same for other words as well)

In [7]:
import re


def split_low_list(lst):
    split_and_lower_fn = lambda s: [e.lower() for e in re.split(' ja |[^\w]|_', s) if e]
    ret = [j for i in [split_and_lower_fn(s) for s in lst] for j in i]
    return ret


def restore_wcs(lst):
    lst1 = lst.copy()
    eri_wc_given = False
    if 'erilliset' in lst:
        lst1.remove('erilliset')
        eri_wc_given = True
    if 'erillinen' in lst:
        lst1.remove('erillinen')
        eri_wc_given = True
    if eri_wc_given:
        lst1.remove('wc')
        lst1 += ['erillinen wc']
    return lst1


def remove_ending(lst):
    lst1 = lst.copy(
    )  # copying and working on that is needed because otherwise remove() messes things up
    for s in lst:
        if ('eet' in s):
            lst1.remove(s)
            lst1 += [s[:-2]]
        if ('sit' in s) or ('vut' in s) or ('vat' in s) or ('lat' in s) or ('mat' in s) or\
            ('mät' in s) or ('han' in s) or ('kut' in s) or ('iön' in s):
            lst1.remove(s)
            lst1 += [s[:-1]]
        if 'erran' in s:
            lst1.remove(s)
            lst1 += [s[:-5] + 'erta']
        if 'uvia' in s:
            lst1.remove(s)
            lst1 += [s[:-2] + 'a']
    return lst1


def correct_typos(lst):
    lst1 = lst.copy()
    for s in lst:
        if s == 'kylphuone':
            lst1.remove(s)
            lst1 += ['kylpyhuone']
        if s == 'makuuhuonee':
            lst1.remove(s)
            lst1 += ['makuuhuone']
        if s == 'makuuhuonet':
            lst1.remove(s)
            lst1 += ['makuuhuone']
        if (s == 'parvekke') or (s == 'pareke'):
            lst1.remove(s)
            lst1 += ['parveke']
    return lst1


def remove_ts(lst):
    lst1 = lst.copy()
    if 't' in lst:
        lst1.remove('t')
    return lst1


def deal_with_saunas(lst):
    lst1 = lst.copy()
    for s in lst:
        if 'sauna' in s:
            lst1.remove(s)
            lst1 += ['sauna']
        if s == 'terassiparveke':
            lst1.remove(s)
            lst1 += ['terassi', 'parveke']
        if s == 'kattoterassi':
            lst1.remove(s)
            lst1 += ['terassi', 'katto']
        if (s == 'autotalli'):
            lst1.remove(s)
            lst1 += ['autohalli']
        if (s == 'allaosasto') or (s == 'allasosasto'):
            lst1.remove(s)
            lst1 += ['allas']
    return lst1

In [8]:
df['room_type_list_prcsd'] = df.room_type_list.apply(split_low_list)
df.room_type_list_prcsd = df.room_type_list_prcsd.apply(restore_wcs)
df.room_type_list_prcsd = df.room_type_list_prcsd.apply(remove_ending)
df.room_type_list_prcsd = df.room_type_list_prcsd.apply(correct_typos)
df.room_type_list_prcsd = df.room_type_list_prcsd.apply(remove_ts)
df.room_type_list_prcsd = df.room_type_list_prcsd.apply(deal_with_saunas)
df.head()

,img_url,room_type,room_type_list,room_type_list_prcsd
0,https://blok-production.imgix.net/photos/e7a3e...,Kylpyhuone,[Kylpyhuone],[kylpyhuone]
1,https://blok-production.imgix.net/photos/e7a3e...,Kylpyhuone,[Kylpyhuone],[kylpyhuone]
2,https://blok-production.imgix.net/photos/e7a3e...,Pohjakuva,[Pohjakuva],[pohjakuva]
3,https://blok-production.imgix.net/photos/e7a3e...,Keittiö,[Keittiö],[keittiö]
4,https://blok-production.imgix.net/photos/e7a3e...,Keittiö,[Keittiö],[keittiö]


Do the same analysis of unique room labels as above:

In [9]:
room_names = pd.Series([e for l in df.room_type_list_prcsd.values for e in l])
unique = room_names.unique()  # all the roomnames that are in the list
unique.sort()
unique

array(['alakerta', 'alempi', 'alkovi', 'allas', 'askarteluhuone',
       'asunto', 'aula', 'autohalli', 'avokeittiö', 'erillinen wc',
       'eteinen', 'etu', 'etupiha', 'huone', 'ilmakuva', 'julkisivu',
       'katto', 'keittiö', 'keittokomero', 'kellari', 'kerros',
       'kesäkuva', 'kesänäkymä', 'kodinhoitohuone', 'kodinhoitotila',
       'kokoustila', 'kuisti', 'kuntosali', 'kuva', 'kylpyhuone',
       'käynnissä', 'käytävä', 'lasitett', 'leikkipaikka', 'lisätila',
       'lukunurkkaus', 'maisema', 'makuu', 'makuuhuone', 'makuutila',
       'myyjän', 'näkymä', 'näköala', 'oleskelutila', 'olohuone',
       'omistajan', 'osittai', 'ottama', 'parkkihalli', 'parveke',
       'parvi', 'patio', 'pesutupa', 'piha', 'pihapiiri', 'pohjakuva',
       'pohjapiirros', 'portaikko', 'ranskalainen', 'ranta',
       'rappukäytävä', 'ruokahuone', 'ruokailutila', 'sauna',
       'sisäänkäynti', 'suihku', 'takapiha', 'takkahuone', 'talo',
       'talopesula', 'taloyhtiö', 'terassi', 'tila', 'työhuon

In [10]:
common_labels = room_names.value_counts()[room_names.value_counts() > 10]
print(common_labels)
print('\nTotal number of labels:', len(common_labels))

olohuone           3304
keittiö            2547
eteinen            2212
kylpyhuone         2075
makuuhuone         2033
parveke            1994
talo               1844
julkisivu          1381
sauna              1337
terassi            1091
erillinen wc       1010
pohjakuva           856
ruokailutila        747
piha                525
yläkerta            269
avokeittiö          194
alakerta            152
huone               134
vaatehuone          130
kodinhoitohuone     110
wc                   91
alkovi               71
takkahuone           68
lasitett             66
taloyhtiö            65
tila                 29
kesäkuva             26
keittokomero         24
omistajan            24
allas                23
takapiha             22
ottama               21
aula                 20
kuva                 19
uima                 19
yhtiö                18
näkymä               16
työhuone             15
ulkokuva             15
maisema              15
ruokahuone           15
parvi           

Now change all labels that occur 67 or less times in the df to 'other'. Reasons:
1. Many of the ones with frequencies 10 or less are probably typos or other forms of things above (like 'lounge' instead of 'living room'), so should probably not even be a distinct class
2. The classes which would have had so few pictures would not have enough pictures to train the model on them (with only say 10 pictures for some type of room the model will have difficulties to generalize from these pictures). 67 is an arbitrary limit I've drawn here (because the two classes below 67 apparently make no sense as words). Probably that limit can also be tested/tuned later on

Note: 'other' might contain many many different kinds of pictures and should therefore maybe excluded from training the model. Needs to be tested then.

In [11]:
rare_labels = room_names.value_counts()[
    room_names.value_counts() <= 67].index.values


def del_rare_labels(lst):
    lst1 = lst.copy()
    add_other = False
    for s in lst:
        if s in rare_labels:
            add_other = True
            lst1.remove(s)
    if add_other:
        lst1 += ['other']
    return lst1


df.room_type_list_prcsd = df.room_type_list_prcsd.apply(del_rare_labels)

In [12]:
room_names = pd.Series([e for l in df.room_type_list_prcsd.values for e in l])
room_names.unique()  # all the roomnames that are in the list

array(['kylpyhuone', 'pohjakuva', 'keittiö', 'parveke', 'julkisivu',
       'olohuone', 'eteinen', 'makuuhuone', 'terassi', 'talo',
       'erillinen wc', 'sauna', 'ruokailutila', 'wc', 'yläkerta', 'piha',
       'alakerta', 'vaatehuone', 'other', 'avokeittiö', 'alkovi', 'huone',
       'kodinhoitohuone', 'takkahuone'], dtype=object)

In [13]:
common_labels = room_names.value_counts()
print(common_labels)
print('\nTotal number of labels:', len(common_labels))

olohuone           3304
keittiö            2547
eteinen            2212
kylpyhuone         2075
makuuhuone         2033
parveke            1994
talo               1844
julkisivu          1381
sauna              1337
terassi            1091
erillinen wc       1010
pohjakuva           856
ruokailutila        747
other               542
piha                525
yläkerta            269
avokeittiö          194
alakerta            152
huone               134
vaatehuone          130
kodinhoitohuone     110
wc                   91
alkovi               71
takkahuone           68
dtype: int64

Total number of labels: 24


We end up with 24 labels!

Should 'wc' and 'erillinen wc' be added together (by converting all 'erillinen wc' to 'wc')? The thing is that blok might want to distinguish between the two, but the pictures probably look the same.

The maximum amount of labels attached to a single image now:

In [14]:
max([len(lst) for lst in df.room_type_list_prcsd.values])

4

Finally translate Finnish labels to English and give them integer values:

In [15]:
df.head()

,img_url,room_type,room_type_list,room_type_list_prcsd
0,https://blok-production.imgix.net/photos/e7a3e...,Kylpyhuone,[Kylpyhuone],[kylpyhuone]
1,https://blok-production.imgix.net/photos/e7a3e...,Kylpyhuone,[Kylpyhuone],[kylpyhuone]
2,https://blok-production.imgix.net/photos/e7a3e...,Pohjakuva,[Pohjakuva],[pohjakuva]
3,https://blok-production.imgix.net/photos/e7a3e...,Keittiö,[Keittiö],[keittiö]
4,https://blok-production.imgix.net/photos/e7a3e...,Keittiö,[Keittiö],[keittiö]


In [16]:
eng_labels = [
    "Living room", "Kitchen", "Hallway", "Bathroom", "Bedroom", "Balcony",
    "House", "Facade", "Sauna", "Terrace", "Separate Wc", "Floor plan",
    "Dining space", "Yard", "Other", "Upstairs", "Open kitchen", "Downstairs",
    "Room", "Clothes room", "Utility room", "Wc", "Alcove", "Fireplace"
]

In [17]:
translations = dict(zip(list(common_labels.index), eng_labels))
word_to_int = dict(zip(eng_labels, list(range(len(eng_labels)))))


def fin_to_eng(lst):
    lst2 = lst.copy()
    lst2 = map(translations.get, lst2, lst2)
    return list(lst2)


def remove_useless_labels(lst):
    lst2 = lst.copy()
    lst2 = [e for e in lst2 if e != '- remove']
    return lst2


def word_to_int_map(lst):
    lst2 = lst.copy()
    lst2 = map(word_to_int.get, lst2, lst2)
    return list(lst2)


df['rooms_eng'] = df.room_type_list_prcsd.apply(fin_to_eng)
df.rooms_eng = df.rooms_eng.apply(remove_useless_labels)
df['labels'] = df.rooms_eng.apply(word_to_int_map)

In [23]:
room_names = pd.Series([e for l in df.labels.values for e in l])
unique = room_names.unique()  # all the roomnames that are in the list
#unique.sort()
np.sort(unique)
unique

array([ 3, 11,  1,  5,  7,  0,  2,  4,  9,  6, 10,  8, 12, 21, 15, 14, 17,
       19, 13, 16, 22, 18, 20, 23])

In [19]:
word_to_int

{'Living room': 0,
 'Kitchen': 1,
 'Hallway': 2,
 'Bathroom': 3,
 'Bedroom': 4,
 'Balcony': 5,
 'House': 6,
 'Facade': 7,
 'Sauna': 8,
 'Terrace': 9,
 'Separate Wc': 10,
 'Floor plan': 11,
 'Dining space': 12,
 'Yard': 13,
 'Other': 14,
 'Upstairs': 15,
 'Open kitchen': 16,
 'Downstairs': 17,
 'Room': 18,
 'Clothes room': 19,
 'Utility room': 20,
 'Wc': 21,
 'Alcove': 22,
 'Fireplace': 23}

In [20]:
df.head(20)

,img_url,room_type,room_type_list,room_type_list_prcsd,rooms_eng,labels
0,https://blok-production.imgix.net/photos/e7a3e...,Kylpyhuone,[Kylpyhuone],[kylpyhuone],[Bathroom],[3]
1,https://blok-production.imgix.net/photos/e7a3e...,Kylpyhuone,[Kylpyhuone],[kylpyhuone],[Bathroom],[3]
2,https://blok-production.imgix.net/photos/e7a3e...,Pohjakuva,[Pohjakuva],[pohjakuva],[Floor plan],[11]
3,https://blok-production.imgix.net/photos/e7a3e...,Keittiö,[Keittiö],[keittiö],[Kitchen],[1]
4,https://blok-production.imgix.net/photos/e7a3e...,Keittiö,[Keittiö],[keittiö],[Kitchen],[1]
5,https://blok-production.imgix.net/photos/e7a3e...,Parveke & Julkisivu,"[Parveke, Julkisivu]","[parveke, julkisivu]","[Balcony, Facade]","[5, 7]"
6,https://blok-production.imgix.net/photos/e7a3e...,Parveke & Julkisivu,"[Parveke, Julkisivu]","[parveke, julkisivu]","[Balcony, Facade]","[5, 7]"
7,https://blok-production.imgix.net/photos/e7a3e...,Parveke & Julkisivu,"[Parveke, Julkisivu]","[parveke, julkisivu]","[Balcony, Facade]","[5, 7]"
8,https://blok-production.imgix.net/photos/e7a3e...,Olohuone & Eteinen,"[Olohuone, Eteinen]","[olohuone, eteinen]","[Living room, Hallway]","[0, 2]"
9,https://blok-production.imgix.net/photos/e7a3e...,Olohuone & Eteinen,"[Olohuone, Eteinen]","[olohuone, eteinen]","[Living room, Hallway]","[0, 2]"


Store the dataframe with processed labels (remove all unnecessary columns):

In [21]:
df2 = df.drop(
    ['room_type', 'room_type_list', 'room_type_list_prcsd', 'rooms_eng'],
    axis=1)
df2.head()

,img_url,labels
0,https://blok-production.imgix.net/photos/e7a3e...,[3]
1,https://blok-production.imgix.net/photos/e7a3e...,[3]
2,https://blok-production.imgix.net/photos/e7a3e...,[11]
3,https://blok-production.imgix.net/photos/e7a3e...,[1]
4,https://blok-production.imgix.net/photos/e7a3e...,[1]


In [22]:
df2.to_pickle('data/prcsd_labels_df')